In [29]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
import os
import shutil
import random

In [33]:
os.chdir('data/sign-lang')
if os.path.isdir('train/0/') is False:
    os.makedirs('train')
    os.makedirs('valid')
    os.makedirs('test')
    for i in range(0,10):
        shutil.move(f'{i}','train')
        os.makedirs(f'valid/{i}')
        os.makedirs(f'test/{i}')
        valid_samples=random.sample(os.listdir(f'train/{i}'),30)
        for c in valid_samples:
            shutil.move(f'train/{i}/{c}',f'valid/{i}')
        test_samples=random.sample(os.listdir(f'train/{i}'),5)
        for c in test_samples:
            shutil.move(f'train/{i}/{c}',f'test/{i}')
os.chdir('../../')

In [12]:
train_path='/Users/sarthak/Desktop/Important/Machine learning/data/sign-lang/train'
valid_path='/Users/sarthak/Desktop/Important/Machine learning/data/sign-lang/valid'
test_path='/Users/sarthak/Desktop/Important/Machine learning/data/sign-lang/test'

In [13]:
train_batch=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path,target_size=(224,224),batch_size=10)
valid_batch=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path,target_size=(224,224),batch_size=10)
test_batch=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path,target_size=(224,224),batch_size=10,shuffle=False)

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [14]:
mobile=tf.keras.applications.mobilenet.MobileNet()

In [15]:
x=mobile.layers[-6].output
output=Dense(units=10,activation='softmax')(x)

In [16]:
model=Model(inputs=mobile.inputs,outputs=output)

In [17]:
for layer in model.layers[:-23]:
    layer.trainable=False

In [18]:
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(x=train_batch,validation_data=valid_batch,epochs=30,verbose=2)

Train for 172 steps, validate for 30 steps
Epoch 1/30
172/172 - 82s - loss: 0.7784 - accuracy: 0.7605 - val_loss: 1.1791 - val_accuracy: 0.5833
Epoch 2/30
172/172 - 76s - loss: 0.1066 - accuracy: 0.9895 - val_loss: 0.9274 - val_accuracy: 0.6967
Epoch 3/30
172/172 - 75s - loss: 0.0386 - accuracy: 1.0000 - val_loss: 0.7753 - val_accuracy: 0.7467
Epoch 4/30
172/172 - 76s - loss: 0.0213 - accuracy: 1.0000 - val_loss: 0.7298 - val_accuracy: 0.7600
Epoch 5/30
172/172 - 77s - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.6819 - val_accuracy: 0.7700
Epoch 6/30
172/172 - 77s - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.6227 - val_accuracy: 0.8067
Epoch 7/30
172/172 - 75s - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.5815 - val_accuracy: 0.8200
Epoch 8/30
172/172 - 75s - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.5846 - val_accuracy: 0.8067
Epoch 9/30
172/172 - 75s - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.5648 - val_accuracy: 0.8167
Epoch 10/30
172/172 - 76s - loss: 0.0038 - accur

In [22]:
p=model.predict(x=test_batch,verbose=2)

5/5 - 2s


In [23]:
cm=confusion_matrix(y_true=test_batch.classes,y_pred=np.argmax(p,axis=-1))

In [25]:
print(cm.trace()/cm.sum()*100)

88.0
